# Multiple Linear Regression

## Import all relevant and necessary libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.feature_selection import RFE

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

## Reading and Comprehending the data

In [ ]:
bike_dataframe = pd.read_csv(r'D:\UPGRAD\Case_Study\BoomBikes\day.csv')
bike_dataframe.head()

In [ ]:
# check for shape of the dataset
bike_dataframe.shape

In [ ]:
#gather information about data
#Checking for count and the type of data present in the given dataset
bike_dataframe.info()

In [ ]:
bike_dataframe.describe()

In [ ]:
bike_dataframe.size

## Data Cleaning

In [ ]:
# check for missing values
bike_dataframe.isnull().sum()

### As part of data cleaning, dropping following columns
- instant: since it is merely index of every record in the dataset
- dteday: this information of date is also available in year, month columns
- casual and registered: since 'cnt' column has summation of these two values

In [ ]:
bike_dataframe.drop(['instant','dteday','casual','registered'],axis=1,inplace=True)

In [ ]:
# dropping the duplicates
bike_dataframe.drop_duplicates(inplace=True)

In [ ]:
bike_dataframe.info()

### As the shape remains after dropping duplicates, which indicates that there are no duplicates in the dataframe

# Changing categorical data which were primarily numeric to more meaningful one

In [ ]:
# Encoding/mapping the season column

bike_dataframe.season = bike_dataframe.season.map({1:'spring', 2:'summer', 3:'fall', 4:'winter'})

# Encoding/mapping the month column

bike_dataframe.mnth = bike_dataframe.mnth.map({1:'jan',2:'feb',3:'mar',4:'apr',5:'may',6:'june',7:'july',8:'aug',9:'sep',10:'oct',11:'nov',12:'dec'})

# Encoding/mapping the weekday column

bike_dataframe.weekday = bike_dataframe.weekday.map({0:'sun',1:'mon',2:'tue',3:'wed',4:'thu',5:'fri',6:'sat'})

# Encoding/mapping the weathersit column

bike_dataframe.weathersit = bike_dataframe.weathersit.map({1:'Clear',2:'Misty',3:'Light_snowrain',4:'Heavy_snowrain'})

bike_dataframe.head()

# Visualizing the data

In [ ]:
# Analysing/visualizing the categorical columns
# to see how predictor variable stands against the target variable

plt.figure(figsize=(20, 12))
plt.subplot(2,4,1)
sns.boxplot(x = 'season', y = 'cnt', data = bike_dataframe)
plt.subplot(2,4,2)
sns.boxplot(x = 'mnth', y = 'cnt', data = bike_dataframe)
plt.subplot(2,4,3)
sns.boxplot(x = 'weekday', y = 'cnt', data = bike_dataframe)
plt.subplot(2,4,4)
sns.boxplot(x = 'weathersit', y = 'cnt', data = bike_dataframe)
plt.subplot(2,4,5)
sns.boxplot(x = 'holiday', y = 'cnt', data = bike_dataframe)
plt.subplot(2,4,6)
sns.boxplot(x = 'workingday', y = 'cnt', data = bike_dataframe)
plt.subplot(2,4,7)
sns.boxplot(x = 'yr', y = 'cnt', data = bike_dataframe)
plt.show()

In [ ]:
# function to create barplot related to categorical columns

def plt_catgy_columns(column):
    plt.figure(figsize = (12,6))
    plt.subplot(1,2,1)
    sns.barplot(x=column,y='cnt',data=bike_dataframe)
    plt.subplot(1,2,2)
    sns.barplot(x=column,y='cnt',data=bike_dataframe, hue='yr',palette='Set1')
    plt.legend(labels=['2018', '2019'])
    plt.show()

In [ ]:
# plotting visualization for season column

plt_catgy_columns('season')

### Fall seems to have more traction to booking. Furthermore, in each season, count of bookings has increased rapidly from 2018 to 2019

In [ ]:
# plotting visualization for month column

plt_catgy_columns('mnth')

### Maximum of bookings has been done between the months of May to Oct, following which the booking trend has decreased until the end of the year. And comparatively number of bookings have tremendously increased in the year 2019 from 2018.

In [ ]:
# plotting visualization for weathersit column

plt_catgy_columns('weathersit')

### From the above graphs, it seems obvious that clear weather attracted more bookings, while standing with same conclusion that with each weather state, 2019 has seen more number of bookings than that of 2018.

In [ ]:
# plotting visualization for weekday column

plt_catgy_columns('weekday')

### It is natural to notice to conclude that end of the week - Thurs, Fri and Sat, sees more bookings than rest of the days, same goes with the conclusion in the year 2019

In [ ]:
# plotting visualization for holiday column

plt_catgy_columns('holiday')

### It is very evident that bookings have enormous increase in its count on holidays

In [ ]:
# plotting visualization for workingday column

plt_catgy_columns('workingday')

### There is slight increase with bookings on the working day, which may imply that customers might prefer to pick a bike for their work more.

In [ ]:
# plotting visualization for year column

plt_catgy_columns('yr')

### Clear evidence that year 2019 has attracted more bookings than year 2018

In [ ]:
# Analysing/visualizing the numerical columns

sns.pairplot(data=bike_dataframe,vars=['temp','atemp','hum','windspeed','cnt'])
plt.show()

In [ ]:
# Checking the correlation between the numerical variables

plt.figure(figsize = (6,6))
matrix = np.triu(bike_dataframe[['temp','atemp','hum','windspeed','cnt']].corr())
sns.heatmap(bike_dataframe[['temp','atemp','hum','windspeed','cnt']].corr(), annot = True, cmap="RdYlGn", mask=matrix)
plt.title("Correlation between Numerical Variables")
plt.show()

### The realtionship between 'temp' and 'atemp' are highly linear and hence these two parameters cannot be used together in model building due to multicollinearity.

# Data Preparation

In [ ]:
# Dummy variable creation for month, weekday, weathersit and season variables.

months_df=pd.get_dummies(bike_dataframe.mnth,drop_first=True)
weekdays_df=pd.get_dummies(bike_dataframe.weekday,drop_first=True)
weathersit_df=pd.get_dummies(bike_dataframe.weathersit,drop_first=True)
seasons_df=pd.get_dummies(bike_dataframe.season,drop_first=True)

bike_dataframe.head()

In [ ]:
# Merging  the dataframe, with the dummy variable dataset. 

df_new = pd.concat([bike_dataframe,months_df,weekdays_df,weathersit_df,seasons_df],axis=1)

# dropping unnecessary columns as we have already created dummy variable out of it.

df_new.drop(['season','mnth','weekday','weathersit'], axis = 1, inplace = True)

df_new.head()

In [ ]:
df_new.info()

# Splitting new dataframe into Train and Test sets

In [ ]:
# splitting the dataframe into Train and Test

np.random.seed(0)
df_train, df_test = train_test_split(df_new, train_size = 0.7, random_state = 100)

In [ ]:
# check the shape of training datatset

df_train.shape

In [ ]:
# check the shape of testing datatset

df_test.shape

In [ ]:
# Using MinMaxScaler to Rescaling the features

scaler = MinMaxScaler()

In [ ]:
# verifying the head of dataset before scaling.

df_train.head()

In [ ]:
# Apply scaler() to all the columns except the 'yes-no' and 'dummy' variables

num_vars = ['temp','atemp','hum','windspeed','cnt']
df_train[num_vars] = scaler.fit_transform(df_train[num_vars])

In [ ]:
# verifying the head after appying scaling.

df_train.head()

In [ ]:
# describing the dataset

df_train.describe()

In [ ]:
# check the correlation coefficients to see which variables are highly correlated

plt.figure(figsize = (25,25))
matrix = np.triu(df_train.corr())
sns.heatmap(df_train.corr(), annot = True, cmap="RdYlGn", mask=matrix)
plt.show()

#### cnt seems to have correlation with year variable and temp. Similarly, Misty and humidity show correlation. Spring season with Jan and Feb month, Summer season with may month and Winter season with oct and nov month show good correlation.

In [ ]:
# Visualizing one of the correlation to see the trends via Scatter plot.

plt.figure(figsize=[6,6])
plt.scatter(df_train.temp, df_train.cnt)
plt.show()

#### the above plot confirms positive correlation between temp and cnt

In [ ]:
# Building the Linear Model

y_train = df_train.pop('cnt')
X_train = df_train

In [ ]:
# Recursive feature elimination 

ridgem = Ridge()
ridgem.fit(X_train, y_train)

rfe = RFE(ridgem,n_features_to_select=15)
rfe = rfe.fit(X_train, y_train)

# Get the selected features
selected_features = rfe.support_

# Print the selected features
print("Selected Features:", selected_features)

In [ ]:
#Columns selected by RFE and their weights
list(zip(X_train.columns,rfe.support_,rfe.ranking_))

In [ ]:
# selecting the selected variable via RFE in col list

col = X_train.columns[rfe.support_]
# col = ['yr', 'holiday', 'temp', 'atemp', 'hum', 'windspeed', 'dec', 'jan',
       # 'july', 'nov', 'sep', 'Light_snowrain', 'Misty', 'spring', 'winter']
print(col)

In [ ]:
# checking which columns has been rejected

X_train.columns[~rfe.support_]

In [ ]:
# Generic function to calculate VIF of variables

def calculateVIF(df):
    # df1 = X_train[cols]
    vif = pd.DataFrame()
    vif['Features'] = df.columns
    vif['VIF'] = [variance_inflation_factor(df.values.astype(float), i) for i in range(df.shape[1])]
    vif['VIF'] = round(vif['VIF'], 2)
    vif = vif.sort_values(by = "VIF", ascending = False)
    return vif 

In [ ]:
# dataframe with RFE selected variables

X_train_rfe = X_train[col]

In [ ]:
# calculate VIF

calculateVIF(X_train_rfe)

#### humidity shows high VIF value

# Building a Linear Model

In [ ]:
# Building 1st linear regression model

X_train_lm_1 = sm.add_constant(X_train_rfe)
lr_1 = sm.OLS(y_train,X_train_lm_1.astype(float)).fit()
print(lr_1.summary())

In [ ]:
# As humidity shows high VIF values hence we can drop it
X_train_new = X_train_rfe.drop(['hum'], axis = 1)

# Run the function to calculate VIF for the new model
calculateVIF(X_train_new)

### Checking further if any more features can be dropped

In [ ]:
# Building 2nd linear regression model

X_train_lm_2 = sm.add_constant(X_train_new)
lr_2 = sm.OLS(y_train,X_train_lm_2.astype(float)).fit()
print(lr_2.summary())

In [ ]:
# We can drop nov variable as it has high p-value
X_train_new = X_train_new.drop(['nov'], axis = 1)

# Run the function to calculate VIF for the new model
calculateVIF(X_train_new)

#### After dropping two features, there is no change in the VIF. Hence proceeding with further analysis

In [ ]:
# Building 3rd linear regression model

X_train_lm_3 = sm.add_constant(X_train_new)
lr_3 = sm.OLS(y_train,X_train_lm_3.astype(float)).fit()
print(lr_3.summary())

In [ ]:
# We can drop dec variable as it has high p-value
X_train_new = X_train_new.drop(['dec'], axis = 1)

# Run the function to calculate VIF for the new model
calculateVIF(X_train_new)

In [ ]:
# Building 4th linear regression model

X_train_lm_4 = sm.add_constant(X_train_new)
lr_4 = sm.OLS(y_train,X_train_lm_4.astype(float)).fit()
print(lr_4.summary())

In [ ]:
# We can drop jan variable as it has high p-value
X_train_new = X_train_new.drop(['jan'], axis = 1)

# Run the function to calculate VIF for the new model
calculateVIF(X_train_new)

In [ ]:
# Building 5th linear regression model

X_train_lm_5 = sm.add_constant(X_train_new)
lr_5 = sm.OLS(y_train,X_train_lm_5.astype(float)).fit()
print(lr_5.summary())

In [ ]:
# We can drop july variable as it has high p-value
X_train_new = X_train_new.drop(['july'], axis = 1)

# Run the function to calculate VIF for the new model
calculateVIF(X_train_new)

#### VIF seems to feasible now with all values below 5

In [ ]:
# Building 6th linear regression model

X_train_lm_6 = sm.add_constant(X_train_new)
lr_6 = sm.OLS(y_train,X_train_lm_6.astype(float)).fit()
print(lr_6.summary())

#### We can cosider the above model i.e lr_6, as it seems to have very low multicolinearity between the predictors and the p-values for all the predictors seems to be significant.

#### F-Statistics value of 248.4 (which is greater than 1) and the p-value of 1.47e-186 i.e almost equals to zero, states that the overall model is significant

In [ ]:
# Checking the parameters and their coefficient values
lr_6.params

# Residual Analysis of Train data and validation

In [ ]:
y_train_pred = lr_6.predict(X_train_lm_6)

### Normality of error terms

In [ ]:
# Plot the histogram of the error terms

fig = plt.figure()
sns.distplot((y_train - y_train_pred), bins = 20)
fig.suptitle('Error Terms', fontsize = 20) 
plt.xlabel('Errors', fontsize = 18)

#### by observation, error terms is following nomral distribution

## Multicollinearity

In [ ]:
calculateVIF(X_train_new)

In [ ]:
plt.figure(figsize=(15,8))
sns.heatmap(X_train_new.corr(),annot = True, cmap="RdYlGn")
plt.show()

#### Clear from heat map that there is no multicollinearity

## Linearity

In [ ]:
# Linear relationship validation using CCPR plot
# Component and component plus residual plot

sm.graphics.plot_ccpr(lr_6, 'temp')
plt.show()

sm.graphics.plot_ccpr(lr_6, 'sep')
plt.show()

sm.graphics.plot_ccpr(lr_6, 'windspeed')
plt.show()

#### Homoscedasticity

In [ ]:
y_train_pred = lr_6.predict(X_train_lm_6)
residual = y_train - y_train_pred
sns.scatterplot(x=y_train, y=residual)
plt.plot(y_train,(y_train - y_train), '-r')
plt.xlabel('Count')
plt.ylabel('Residual')
plt.show()

No visible pattern observed from above plot for residuals.

#### Independence of residuals

Durbin-Watson value of final model lr_6 is 2.085, which signifies there is no autocorrelation.

## Step 7: Making Predictions Using the Final Model

Now that we have fitted the model and checked the normality of error terms, it's time to go ahead and make predictions using the final, i.e. 6th model.

In [ ]:
# Applying scaling on the test dataset

num_vars = ['temp', 'atemp', 'hum', 'windspeed','cnt']
df_test[num_vars] = scaler.transform(df_test[num_vars])
df_test.head()

In [ ]:
df_test.describe()

In [ ]:
y_test = df_test.pop('cnt')
X_test = df_test

In [ ]:
col1 = X_train_new.columns

X_test = X_test[col1]

# Adding constant variable to test dataframe
X_test_lm_6 = sm.add_constant(X_test)

In [ ]:
y_pred = lr_6.predict(X_test_lm_6)

In [ ]:
r2 = r2_score(y_test, y_pred)
round(r2,4)

## Step 8: Model Evaluation
plot the graph for actual versus predicted values.

In [ ]:
# Plotting y_test and y_pred to understand the spread

fig = plt.figure()
plt.scatter(y_test, y_pred)
fig.suptitle('y_test vs y_pred', fontsize = 20) 
plt.xlabel('y_test', fontsize = 18)
plt.ylabel('y_pred', fontsize = 16) 

In [ ]:
round(lr_6.params,4)

Hence, the equation of best fitted line is:

$ cnt = 0.1909 + 0.2341  *  year - 0.0963  *  holiday + 0.4777 * temp - 0.1481 * windspeed + 0.0910 * sep - 0.2850 * Light_snowrain - 0.0787 * Misty - 0.0554 * spring + 0.0621 * summer + 0.0945 * winter $

In [ ]:
# Calculating Adjusted-R^2 value for the test dataset

adjusted_r2 = round(1-(1-r2)*(X_test.shape[0]-1)/(X_test.shape[0]-X_test.shape[1]-1),4)
print(adjusted_r2)

In [ ]:
# Visualizing the fit on the test data
# plotting a Regression plot

plt.figure()
sns.regplot(x=y_test.astype(float), y=y_pred.astype(float), ci=68, fit_reg=True,scatter_kws={"color": "blue"}, line_kws={"color": "red"})
plt.title('y_test vs y_pred', fontsize=20)
plt.xlabel('y_test', fontsize=18)
plt.ylabel('y_pred', fontsize=16)
plt.show()

# Training and Testing dataset comparison:
    - Train dataset R^2          : 0.833
    - Test dataset R^2           : 0.8038
    - Train dataset Adjusted R^2 : 0.829    
    - Test dataset Adjusted R^2  : 0.7944

#### The real Factors which are attracting the demand of bikes are year, holiday, temp, windspeed, sep, Light_snowrain, Misty, spring, summer and winter. 